### Forecasting the number of orders based on the international competition Global Manafement Challenge

# Table of contents

1. Abstract
2. About dataset
3. Import librariers
4. Prepare data
5. Exploratory data
6. Train Test Split
7. Functions
8. Custom Transformers
9. Pipeline
10. Models
11. Summary

# 1. Abstract

It is a simulation of production company management. It is a simulation of the management of a manufacturing company, in which student and company teams participate. The teams play the role of the company management. The company is listed on the stock exchange. The role of the board is to make decisions based on quarterly reports. The companies operate internationally in three markets:
* European Union
* NAFTA - the United States, Canada and Mexico
* Internet

Teams are divided into up to eight teams. They compete to achieve the highest possible investment result at the end of the simulation. 
Each board makes **75 decisions**, including: production, research and development, sales, recruitment, marketing, investments, credits, dividend payment, issue or purchase of shares. The company produces and sells 3 different products in 3 different markets. 

<img src="graphics/01.jpg" width="550">

Each team receives a report:

* Resources and products

<img src="graphics/02.jpg" width="550">
 
* The financial statements and

<img src="graphics/03.jpg" width="550">
 
* Information about all companies in the group, the market situation and also information requested by the team.

<img src="graphics/04.jpg" width="550">

<img src="graphics/05.jpg" width="550">

<img src="graphics/06.jpg" width="550">

<img src="graphics/07.jpg" width="550">

The most important factor is to maximize profit. This is done by selling products. By delivering more products than the customers actually order, we create stocks that we have to store, thus increasing losses, because the products produced by our company do not create added value. Another situation is to produce too few products. Therefore, **the most important thing is to forecast the number of products ordered**.

# 2. About dataset

The data set was created from 257 reports. The reports come from previous editions of my team and from the infected ones. Specific features were selected to build models. They were chosen on the basis of their field knowledge.

The reports inside the dataset are:
    - datasets/
        - 01_reports/
            - reports in xlsx files
        - 02_all_reports_data/
        - 03_all_reports_data_current_previous/
        - 04_demand_datasets/
    - graphics/
    - models/
    - scripts/

Each file has a structured name:

<img src="graphics/08.jpg" width="550">

# 3. Import libraries

Import of the necessery modules

In [ ]:
# error display
import warnings
warnings.filterwarnings('ignore')

# data manipulation
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from matplotlib.ticker import PercentFormatter
%matplotlib inline

# pipeline construction
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

# data processing
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder

# models trainng
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb

# optimization of hyperparameters
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, partial, space_eval
from sklearn.model_selection import cross_val_score

# model evaluation
import sklearn.metrics as metrics

# saving models
import pickle

# settings
pd.set_option('display.max_columns', None)
pio.renderers.default='notebook'

# 4. Prepare data

## 4.1 All  Reports Data

From all reports values were taken and created a file **all_reports_data.csv**

<img src="graphics/09.jpg" width="550">

The file was saved here:
    - datasets/
        - 01_reports/
        - 02_all_reports_data/
            - all_reports_data.csv
        - 03_all_reports_data_current_previous/
        - 04_demand_datasets/
    - graphics/
    - models/
    - scripts/

In [ ]:
%run -i 'scripts/01_prepare_all_reports_data/prepare_all_reports_data.py'

## 4.2 All Reports Data -> current and previous

Adding values from the previous cycle.

The file was saved here:
    - datasets/
        - 01_reports/
        - 02_all_reports_data/
        - 03_all_reports_data_current_previous/
            - all_reports_data_current_previous.csv
        - 04_demand_datasets/
    - graphics/
    - models/
    - scripts/

In [ ]:
%run -i 'scripts/02_prepare_all_reports_current_previous/prepare_all_reports_current_previous.py'

## 4.3 Demand Datasets

In this step, variables that affect the number of orders are taken from **all_reports_data_current_previous.csv**. The data is divided into 9 files depending on the product and area, because for them the variables are a little different. Information which variables to choose and how to prepare them was taken from the [blog](https://gmcworld.org/blog/demand-factors) and based on my own knowledge and experience. The functions were not taken because it was assumed that the model would detect them.

<img src="graphics/10.jpg" width="850">

The files was saved here:
    - datasets/
        - 01_reports/
        - 02_all_reports_data/
        - 03_all_reports_data_current_previous/
        - 04_demand_datasets/
            - Prod1_Europe.csv
            - Prod1_Internet.csv
            - Prod1_Nafta.csv
            - Prod2_Europe.csv
            - Prod2_Internet.csv
            - Prod2_Nafta.csv
            - Prod3_Europe.csv
            - Prod3_Internet.csv
            - Prod3_Nafta.csv
    - graphics/
    - models/
    - scripts/

In [ ]:
%run -i 'scripts/03_prepare_demand_datasets/prepare_demand_datasets.py'

## 4.4 Data connection

Combine 9 files into one common file.

In [ ]:
path = 'datasets/04_demand_datasets/'

### Product 1 Europe

In [ ]:
Prod1_Europe = pd.read_csv(f'{path}Prod1_Europe.csv')
Prod1_Europe.head()

### Product 1 Nafta

In [ ]:
Prod1_Nafta = pd.read_csv(f'{path}Prod1_Nafta.csv')
Prod1_Nafta.head()

### Product 1 Internet

In [ ]:
Prod1_Internet = pd.read_csv(f'{path}Prod1_Internet.csv')
Prod1_Internet.head()

### Product 2 Europe

In [ ]:
Prod2_Europe = pd.read_csv(f'{path}Prod2_Europe.csv')
Prod2_Europe.head()

### Product 2 Nafta

In [ ]:
Prod2_Nafta = pd.read_csv(f'{path}Prod2_Nafta.csv')
Prod2_Nafta.head()

### Product 2 Internet

In [ ]:
Prod2_Internet = pd.read_csv(f'{path}Prod2_Internet.csv')
Prod2_Internet.head()

### Product 3 Europe

In [ ]:
Prod3_Europe = pd.read_csv(f'{path}Prod3_Europe.csv')
Prod3_Europe.head()

### Product 3 Nafta

In [ ]:
Prod3_Nafta = pd.read_csv(f'{path}Prod3_Nafta.csv')
Prod3_Nafta.head()

### Product 3 Internet

In [ ]:
Prod3_Internet = pd.read_csv(f'{path}Prod3_Internet.csv')
Prod3_Internet.head()

In [ ]:
list_of_files = [
    Prod1_Europe, Prod2_Europe, Prod3_Europe,
    Prod1_Nafta, Prod2_Nafta, Prod3_Nafta,
    Prod1_Internet, Prod2_Internet, Prod3_Internet
]

# Areas

# Europe
for file in list_of_files[:3]:
    file['Support'] = 0
    file['FailedVisits'] = 0
    file['WebDev'] = 0
    file['Area'] = 'Europe'

# Nafta
for file in list_of_files[3:6]:
    file['Support'] = 0
    file['FailedVisits'] = 0
    file['WebDev'] = 0
    file['Area'] = 'Nafta'

# Internet
for file in list_of_files[6:]:
    file['Commission'] = 0
    file['AgentsDistr'] = 0
    file['Area'] = 'Internet'

# Product 1
for file in list_of_files[0::3]:
    file['Product'] = 1

# Product 2
for file in list_of_files[1::3]:
    file['Product'] = 2
    
# Product 3
for file in list_of_files[2::3]:
    file['Product'] = 3
    
data = pd.concat(list_of_files)
data.drop(['Team', 'MarketShares_c'], axis=1, inplace=True)
data = data[data['Cycle'] >= 5].reset_index(drop=True)
data = data[data['NumOrders'] >= 10].reset_index(drop=True)
data['History'] = pd.Categorical(data['History'])
data['Cycle'] = pd.Categorical(data['Cycle'])
data['Area'] = pd.Categorical(data['Area'])
data['Product'] = pd.Categorical(data['Product'])

# setting the first column of the explanatory variable
col_name="NumOrders"
first_col = data.pop(col_name)
data.insert(0, col_name, first_col)

data

# 5. Exploratory

In [ ]:
data.info(memory_usage = 'deep')

In [ ]:
data.describe() 

Grouping data by product and area.

In [ ]:
data.groupby(["Product", "Area"]).mean()

A **BOXPLOT** shows the distribution of the data with more detailed information. It shows the outliers more clearly, maximum, minimum, quartile(Q1), third quartile(Q3), interquartile range(IQR), and median.

In [ ]:
for column in data.select_dtypes(include=['int64', 'float64']).columns:
    plt.figure(figsize=(14, 3))
    data_selected = data.copy()
    hue_selected = "Area"
    if column == 'Commission':
        data_selected = data_selected[data_selected['Area'] != 'Internet']
    elif column == 'AgentsDistr':
        data_selected = data_selected[data_selected['Area'] != 'Internet']
    elif column == 'Training':
        hue_selected = None
    elif column == 'ManagBudget':
        hue_selected = None
    elif column == 'RandD':
        hue_selected = None
    elif column == 'AssemblyTime':
        hue_selected = None
    elif column == 'Support':
        data_selected = data_selected[data_selected['Area'] == 'Internet']
    elif column == 'FailedVisits':
        data_selected = data_selected[data_selected['Area'] == 'Internet']
    elif column == 'WebDev':
        data_selected = data_selected[data_selected['Area'] == 'Internet']
    sns.boxplot(x = "Product", y = column, hue = hue_selected, data = data_selected, palette = "Blues")
    plt.xlabel('Product')
    plt.ylabel(f'{column}')
    if column not in ['Training', 'ManagBudget', 'RandD', 'AssemblyTime', 'Support', 'FailedVisits', 'WebDev']:
        plt.legend(loc = 'upper right')
    plt.show()

Some data have outliers, but they are real. This may indicate too few reports.

In [ ]:
len(data.select_dtypes(include=['category']).columns)

In [ ]:
fig = plt.figure(figsize = (16, 4))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for (column, i) in zip(data.select_dtypes(include=['category']).columns, range(1, 5)):
    ax = fig.add_subplot(1, 4, i)
    data[column].value_counts().plot.bar()
    plt.title(f'Column: {column}\nUnique values: {len(data[column].unique())}')
    plt.ylabel('frequency')
plt.show()

There is more data for history number 3 and there is less data with each subsequent cycle.

In [ ]:
fig, axs = plt.subplots(ncols=5, nrows=4, figsize=(16, 16))
fig.subplots_adjust(hspace=0.6, wspace=0.4)
index = 0
axs = axs.flatten()
for k,v in data.select_dtypes(include=['int64', 'float64']).items():
    sns.distplot(v, ax=axs[index]).set_title(f'\nColumn: {k}\nUnique values: {len(data[k].unique())}\n')
    index = index + 1
for empty in range(17, 20):
    axs[empty].axis('off')

The data is not normal distribution. You can see the division into products. Some data are skewed.

But the Models allowed my team to reach the national final.

In [ ]:
for column in data.select_dtypes(include=['int64', 'float64']).columns[1:]:
    print(f'NumOrders ({column})')
    g = sns.FacetGrid(data, col="Product", hue="Area", height = 5)
    g.map(plt.scatter, column, "NumOrders", alpha=.8)
    g.add_legend()
    plt.show()

heatmap

In [ ]:
# Plot
plt.figure(figsize=(16, 11))
ax = sns.heatmap(data.corr(), 
                 xticklabels=data.corr().columns, 
                 yticklabels=data.corr().columns, 
                 cmap='RdYlGn', 
                 center=0, 
                 annot=True)

# Decorations
plt.title('Correlogram of variables', fontsize=22)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
scaled_data = scale(data.select_dtypes(include=['int64', 'float64']).drop(['NumOrders'], axis=1))
pca_explained_data = PCA()
pca_explained_data.fit(scaled_data)
print(np.cumsum(pca_explained_data.explained_variance_ratio_))
plt.figure(figsize=(16, 9))
plt.bar(range(pca_explained_data.n_components_),np.cumsum(pca_explained_data.explained_variance_ratio_))
plt.show()

In [ ]:
sns.lmplot(x="NumSales_p", y="MarketShares_p", 
           data=data[(data['Product']==1)&(data['MarketShares_p']!=0)], 
           fit_reg=True, hue='Area', legend=False)
plt.legend(loc='lower right')
plt.show()

You can find dependence on such variables as **advertising**, **previous orders** or **price**, but the greatest influence has information about what **product** it is and on which **area** it is sold.

# 6. Train Test Split

In [ ]:
# split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(data.drop('NumOrders', axis = 1),
                                                    data['NumOrders'],
                                                    test_size = 0.2,
                                                    stratify = data['Area'],
                                                    random_state = 123)

# 7. Functions

Functions used to evaluate models and save them.

In [ ]:
metrics_data_frame = pd.DataFrame(columns = [
    'RMSE_train', 'Max_error_train', 'MAPE_train', 'RMSE_test', 'Max_error_test', 'MAPE_test'
])

def model_evaluation(model, name,
                     X_test = X_test, X_train = X_train, y_test = y_test, y_train = y_train):
    
    global metrics_data_frame
    
    def hist_of_residuals(X, y, set_name):
        errors = model.predict(X) - y
        plt.hist(errors, bins = 100)
        plt.title(f'Histogram of residuals - {set_name} set')
    
    def plot_of_residuals(X, y, set_name):
        errors = model.predict(X) - y
        plt.scatter(x = y, y = errors)
        plt.axhline(0, color="r", linestyle="--")
        plt.xlabel('True Value')
        plt.ylabel('Residual')
        plt.title(f'Plot of residuals - {set_name} set')
    
    def fit_scatter_plot(X, y, set_name):
        y_fitted_values = model.predict(X)

        xmin = y.min()
        xmax = y.max()
        plt.scatter(x = y_fitted_values, y = y)
        x_line = np.linspace(xmin, xmax, 10)
        y_line = x_line
        plt.plot(x_line, y_line, 'r--')
        plt.xlabel('Prediction')
        plt.ylabel('True Value')
        plt.title(f'Plot of predicted values versus true values - {set_name} set')
        
    def calculate_metrics(model = model, name = name,
                          X_test = X_test, X_train = X_train, y_test = y_test, y_train = y_train):
        
        def mean_absolute_percentage_error(y_true, y_pred): 
            y_true, y_pred = y_true.ravel(), y_pred.ravel()
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        
        # quality of fit
        fitted_values = model.predict(X_train)
        rmse_train = np.sqrt(metrics.mean_squared_error(y_train, fitted_values))
        max_error_train = np.max(np.abs(fitted_values - y_train))
        mape_train = mean_absolute_percentage_error(y_true = y_train,
                                                    y_pred = fitted_values)

        # quality of prediction
        y_pred = model.predict(X_test)
        rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        max_error_test = np.max(np.abs(y_pred - y_test))
        mape_test = mean_absolute_percentage_error(y_true = y_test,
                                                   y_pred = y_pred)
        
        calculated_metrics = pd.DataFrame({
            'RMSE_train': rmse_train, 'Max_error_train':max_error_train,'MAPE_train': mape_train,
            'RMSE_test': rmse_test, 'Max_error_test':max_error_test, 'MAPE_test': mape_test
        }, index = [name])
                
        return calculated_metrics
    
    fig = plt.figure(figsize = (16, 12))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    
    ax = fig.add_subplot(3, 2, 1)
    fit_scatter_plot(X = X_train, y = y_train, set_name = 'train')
    
    ax = fig.add_subplot(3, 2, 2)
    fit_scatter_plot(X = X_test, y = y_test, set_name = 'test')
    
    ax = fig.add_subplot(3, 2, 3)
    plot_of_residuals(X = X_train, y = y_train, set_name = 'train')
    
    ax = fig.add_subplot(3, 2, 4)
    plot_of_residuals(X = X_test, y = y_test, set_name = 'test')    

    ax = fig.add_subplot(3, 2, 5)
    hist_of_residuals(X = X_train, y = y_train, set_name = 'train')   

    ax = fig.add_subplot(3, 2, 6)
    hist_of_residuals(X = X_test, y = y_test, set_name = 'test')
    
    metrics_data_frame = metrics_data_frame.append(calculate_metrics())
    return metrics_data_frame

def save_model(model, name):
    filename = f'models/{name}.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print(f'The model {name} has been saved')

# 8. Custom Transformers

In [ ]:
class CreateDummies(BaseEstimator, TransformerMixin):
    def __init__(self, drop_first = True, columns_to_left = None):
        self.drop_first = drop_first
        self.columns_to_left = columns_to_left
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        X = pd.DataFrame(X)
        if self.columns_to_left != None:
            X_left = X[self.columns_to_left].astype(int).copy()
        X = pd.get_dummies(X, drop_first = self.drop_first).copy()
        if self.columns_to_left != None: 
            X = pd.concat([X, X_left], axis = 1)
        return X

# pokazac na wykresie wyzej zaleznosci dla Missing data
class ImputeMissingMarketShares_p(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = [
            'MarketShares_p', 'NumSales_p', 
            'History_1', 'History_2', 'History_3',
            'Cycle_6', 'Cycle_7', 'Cycle_8', 'Cycle_9',
            'Area_Internet', 'Area_Nafta',
            'Product_2', 'Product_3'
        ]
        self.model = None
    
    def fit(self, X, y = None):
        X_fit = X.copy()
        filer_5_cycle = (X_fit['Cycle_6']==0)&(X_fit['Cycle_7']==0)&(X_fit['Cycle_8']==0)&(X_fit['Cycle_9']==0)
        X_fit[filer_5_cycle]['Cycle_6'] = 1
        X_fit = X_fit[X_fit['MarketShares_p'] != 0][self.columns]
        reg_model = LinearRegression()
        reg_model.fit(X_fit.drop(['MarketShares_p'], axis=1),
                      X_fit['MarketShares_p'])
        self.model = reg_model
        return self
    
    def transform(self, X, y = None):
        reg_model = self.model
        filer_5_cycle = (X['Cycle_6']==0)&(X['Cycle_7']==0)&(X['Cycle_8']==0)&(X['Cycle_9']==0)
        X[filer_5_cycle]['Cycle_6'] = 1
        preds = reg_model.predict(X[X['MarketShares_p'] == 0][self.columns].drop(['MarketShares_p'], axis=1))
        X['MarketShares_p'][X['MarketShares_p'] == 0] = preds
        return X

class RemoveColumnsUintTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        return X.drop(list(X.select_dtypes(include=['uint8']).columns), axis=1)

class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        def log_transform(x):
            return 0 if x <= 0 else np.log(x)
        
        X = pd.DataFrame(X)
        for col in X:
            X[col] = X[col].apply(log_transform)
        return X

# 9. Pipeline

In [ ]:
# sub pipeline categorical
cols_categorical = ['History', 'Cycle', 'Area', 'Product']
transformer_categorical = Pipeline(steps=[
    ('dummies', CreateDummies(drop_first = True, columns_to_left = None))
])

# sub pipeline numerical
cols_numerical = list(X_train.columns)
transformer_numerical = Pipeline(steps=[
    ('create_dummies', CreateDummies(drop_first = True, columns_to_left = ['Cycle'])),
    ('impute_missing_MarketShares_p', ImputeMissingMarketShares_p()),
    ('remove_columns_uint_transformer', RemoveColumnsUintTransformer()),
    ('log_transform', LogTransformer()),
    ('first_scaler', StandardScaler()), # we have 17 columns after this
    ('dim_red', PCA()),
    ('second_scaler', StandardScaler())
])

# transformer = numerical + categorical
transformer = ColumnTransformer(transformers=[
    ('categorical', transformer_categorical, cols_categorical),
    ('numerical', transformer_numerical, cols_numerical)
])

In [ ]:
list(transformer.get_params().keys())

# 10. Models

## 10.1 Baseline Model

In [ ]:
baseline_pipeline = Pipeline(steps=[('preprocessor', transformer),
                                    ('regression', LinearRegression())])

In [ ]:
baseline_pipeline.fit(X_train, y_train)

In [ ]:
save_model(model = baseline_pipeline, name = 'baseline_pipeline')
model_evaluation(model = baseline_pipeline, name = 'baseline_model')

## 10.2 Polynomial Regression

In [ ]:
polynomial_regression_pipeline = Pipeline(steps=[('preprocessor', transformer),
                                                 ('polynomial_features',PolynomialFeatures(degree=3)),
                                                 ('regression_model', ElasticNet(alpha = 0.1, l1_ratio = 0.5))])

In [ ]:
n_startup_jobs = 64
 

max_evals = 128 

space ={
    'preprocessor__numerical__log_transform': hp.choice('preprocessor__numerical__log_transform',
                                                        [LogTransformer(),
                                                         PowerTransformer(method='box-cox')]),
    'preprocessor__numerical__first_scaler': hp.choice('preprocessor__numerical__first_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'preprocessor__numerical__dim_red__n_components': hp.uniform('preprocessor__numerical__dim_red__n_components',
                                                                0.8, 1.0),
    'preprocessor__numerical__second_scaler': hp.choice('preprocessor__numerical__second_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'polynomial_features__degree': hp.choice('polynomial_features__degree', np.arange(1, 4).tolist()),
    'regression_model__alpha': hp.loguniform ('regression_model__alpha', 0.0001, 10.0),
    'regression_model__l1_ratio': hp.uniform ('regression_model__l1_ratio', 0.0, 1.0),
}

def objective(space):
    polynomial_regression_params = {
        'preprocessor__numerical__log_transform': space['preprocessor__numerical__log_transform'],
        'preprocessor__numerical__first_scaler': space['preprocessor__numerical__first_scaler'],
        'preprocessor__numerical__dim_red__n_components': space['preprocessor__numerical__dim_red__n_components'],
        'preprocessor__numerical__second_scaler': space['preprocessor__numerical__second_scaler'],
        'polynomial_features__degree': space['polynomial_features__degree'],
        'regression_model__alpha': space['regression_model__alpha'],
        'regression_model__l1_ratio': space['regression_model__l1_ratio'],
    }
    
    polynomial_regression_pipeline.set_params(**polynomial_regression_params) 
    
    score = - cross_val_score(polynomial_regression_pipeline, X_train, y_train, cv=10,
                            scoring = 'neg_root_mean_squared_error', n_jobs = -1).mean()
    
    return{'loss':score, 'status': STATUS_OK }

trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                   max_evals=max_evals,
                   trials=trials)
 
best_params = space_eval(space, best_params)
print('\nThe best params:')
print ("{:<60} {}".format('Parameter','Selected'))
for k, v in best_params.items():
    print ("{:<60} {}".format(k, v))

polynomial_regression = polynomial_regression_pipeline.set_params(**best_params)
polynomial_regression.fit(X_train, y_train)

In [ ]:
save_model(model = polynomial_regression, name = 'polynomial_regression')
model_evaluation(model = polynomial_regression, name = 'polynomial_regression')

## 10.3 Random Forest Regression

In [ ]:
random_forest_regression_pipeline = Pipeline(steps=[('preprocessor', transformer),
                                                    ('regression_model', RandomForestRegressor(n_jobs = -1))])

In [ ]:
n_startup_jobs = 128
 

max_evals = 256 

space ={
    'preprocessor__numerical__log_transform': hp.choice('preprocessor__numerical__log_transform',
                                                        [LogTransformer(),
                                                         PowerTransformer(method='box-cox')]),
    'preprocessor__numerical__first_scaler': hp.choice('preprocessor__numerical__first_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'preprocessor__numerical__dim_red__n_components': hp.uniform('preprocessor__numerical__dim_red__n_components',
                                                                0.8, 1.0),
    'preprocessor__numerical__second_scaler': hp.choice('preprocessor__numerical__second_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'regression_model__max_depth': hp.choice('regression_model__max_depth', np.arange(2, 51).tolist()),
    'regression_model__min_samples_leaf': hp.choice('regression_model__min_samples_leaf', 
                                                    np.arange(1, 6).tolist()),
    'regression_model__min_samples_split': hp.choice('regression_model__min_samples_split',
                                                     np.arange(2, 11).tolist()),
    'regression_model__max_leaf_nodes': hp.choice('regression_model__max_leaf_nodes', np.arange(5, 31).tolist()),
    'regression_model__n_estimators': hp.choice('regression_model__n_estimators',
                                                np.arange(50, 1050, 50).tolist()),
}

def objective(space):
    random_forest_regression_params = {
        'preprocessor__numerical__log_transform': space['preprocessor__numerical__log_transform'],
        'preprocessor__numerical__first_scaler': space['preprocessor__numerical__first_scaler'],
        'preprocessor__numerical__dim_red__n_components': space['preprocessor__numerical__dim_red__n_components'],
        'preprocessor__numerical__second_scaler': space['preprocessor__numerical__second_scaler'],
        'regression_model__max_depth': space['regression_model__max_depth'],
        'regression_model__min_samples_leaf': space['regression_model__min_samples_leaf'],
        'regression_model__min_samples_split': space['regression_model__min_samples_split'],
        'regression_model__max_leaf_nodes': space['regression_model__max_leaf_nodes'],
        'regression_model__n_estimators': space['regression_model__n_estimators'],
    }
    
    random_forest_regression_pipeline.set_params(**random_forest_regression_params) 
    
    score = - cross_val_score(random_forest_regression_pipeline, X_train, y_train, cv=10,
                              scoring = 'neg_root_mean_squared_error', n_jobs = -1).mean()
    
    return{'loss':score, 'status': STATUS_OK }

trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                   max_evals=max_evals,
                   trials=trials)
 
best_params = space_eval(space, best_params)
print('\nThe best params:')
print ("{:<60} {}".format('Parameter','Selected'))
for k, v in best_params.items():
    print ("{:<60} {}".format(k, v))

random_forest_regression = random_forest_regression_pipeline.set_params(**best_params)
random_forest_regression.fit(X_train, y_train)

In [ ]:
save_model(model = random_forest_regression, name = 'random_forest_regression')
model_evaluation(model = random_forest_regression, name = 'random_forest_regression')

## 10.5 Extra Trees Regression

In [ ]:
extra_trees_regression_pipeline = Pipeline(steps=[('preprocessor', transformer),
                                                  ('regression_model', ExtraTreesRegressor(n_jobs = -1))])

In [ ]:
n_startup_jobs = 128
 

max_evals = 256 

space ={
    'preprocessor__numerical__log_transform': hp.choice('preprocessor__numerical__log_transform',
                                                        [LogTransformer(),
                                                         PowerTransformer(method='box-cox')]),
    'preprocessor__numerical__first_scaler': hp.choice('preprocessor__numerical__first_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'preprocessor__numerical__dim_red__n_components': hp.uniform('preprocessor__numerical__dim_red__n_components',
                                                                0.8, 1.0),
    'preprocessor__numerical__second_scaler': hp.choice('preprocessor__numerical__second_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'regression_model__max_depth': hp.choice('regression_model__max_depth', np.arange(2, 51).tolist()),
    'regression_model__min_samples_leaf': hp.choice('regression_model__min_samples_leaf', 
                                                    np.arange(1, 6).tolist()),
    'regression_model__min_samples_split': hp.choice('regression_model__min_samples_split',
                                                     np.arange(2, 11).tolist()),
    'regression_model__max_leaf_nodes': hp.choice('regression_model__max_leaf_nodes', np.arange(5, 31).tolist()),
    'regression_model__n_estimators': hp.choice('regression_model__n_estimators',
                                                np.arange(50, 1050, 50).tolist()),
}

def objective(space):
    extra_trees_regression_params = {
        'preprocessor__numerical__log_transform': space['preprocessor__numerical__log_transform'],
        'preprocessor__numerical__first_scaler': space['preprocessor__numerical__first_scaler'],
        'preprocessor__numerical__dim_red__n_components': space['preprocessor__numerical__dim_red__n_components'],
        'preprocessor__numerical__second_scaler': space['preprocessor__numerical__second_scaler'],
        'regression_model__max_depth': space['regression_model__max_depth'],
        'regression_model__min_samples_leaf': space['regression_model__min_samples_leaf'],
        'regression_model__min_samples_split': space['regression_model__min_samples_split'],
        'regression_model__max_leaf_nodes': space['regression_model__max_leaf_nodes'],
        'regression_model__n_estimators': space['regression_model__n_estimators'],
    }
    
    extra_trees_regression_pipeline.set_params(**extra_trees_regression_params) 
    
    score = - cross_val_score(extra_trees_regression_pipeline, X_train, y_train, cv=10,
                              scoring = 'neg_root_mean_squared_error', n_jobs = -1).mean()
    
    return{'loss':score, 'status': STATUS_OK }

trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                   max_evals=max_evals,
                   trials=trials)
 
best_params = space_eval(space, best_params)
print('\nThe best params:')
print ("{:<60} {}".format('Parameter','Selected'))
for k, v in best_params.items():
    print ("{:<60} {}".format(k, v))

extra_trees_regression = extra_trees_regression_pipeline.set_params(**best_params)
extra_trees_regression.fit(X_train, y_train)

In [ ]:
save_model(model = extra_trees_regression, name = 'extra_trees_regression')
model_evaluation(model = extra_trees_regression, name = 'extra_trees_regression')

## 10.6 Support Vector Regression

In [ ]:
svr_regression_pipeline = Pipeline(steps=[('preprocessor', transformer),
                                          ('regression_model', SVR(kernel='rbf'))])

In [ ]:
n_startup_jobs = 256
 
max_evals = 1024 

space ={
    'preprocessor__numerical__log_transform': hp.choice('preprocessor__numerical__log_transform',
                                                        [LogTransformer(),
                                                         PowerTransformer(method='box-cox')]),
    'preprocessor__numerical__first_scaler': hp.choice('preprocessor__numerical__first_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'preprocessor__numerical__dim_red__n_components': hp.uniform('preprocessor__numerical__dim_red__n_components',
                                                                0.8, 1.0),
    'preprocessor__numerical__second_scaler': hp.choice('preprocessor__numerical__second_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'regression_model__C': hp.uniform ('regression_model__C', 1.0, 50000.0),
    'regression_model__epsilon': hp.uniform ('regression_model__epsilon', 0.0001, 0.5),
    'regression_model__gamma': hp.uniform ('regression_model__gamma', 0.0001, 100.0)
}

def objective(space):
    svr_regression_params = {
        'preprocessor__numerical__log_transform': space['preprocessor__numerical__log_transform'],
        'preprocessor__numerical__first_scaler': space['preprocessor__numerical__first_scaler'],
        'preprocessor__numerical__dim_red__n_components': space['preprocessor__numerical__dim_red__n_components'],
        'preprocessor__numerical__second_scaler': space['preprocessor__numerical__second_scaler'],
        'regression_model__C': space['regression_model__C'],
        'regression_model__epsilon': space['regression_model__epsilon'],
        'regression_model__gamma': space['regression_model__gamma'],

    }
    
    svr_regression_pipeline.set_params(**svr_regression_params) 
    
    score = - cross_val_score(svr_regression_pipeline, X_train, y_train, cv=10,
                              scoring = 'neg_root_mean_squared_error', n_jobs = -1).mean()
    
    return{'loss':score, 'status': STATUS_OK }

trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                   max_evals=max_evals,
                   trials=trials)
 
best_params = space_eval(space, best_params)
print('\nThe best params:')
print ("{:<60} {}".format('Parameter','Selected'))
for k, v in best_params.items():
    print ("{:<60} {}".format(k, v))

svr_regression = svr_regression_pipeline.set_params(**best_params)
svr_regression.fit(X_train, y_train)

In [ ]:
save_model(model = svr_regression, name = 'svr_regression')
model_evaluation(model = svr_regression, name = 'svr_regression')

## 10.7 AdaBoost Regression

In [ ]:
adaboost_regression_pipeline = Pipeline(steps=[('preprocessor', transformer),
                                          ('regression_model',
                                           AdaBoostRegressor(DecisionTreeRegressor(max_depth=3),
                                                             n_estimators=500, learning_rate=0.5,
                                                             random_state = 123))])

In [ ]:
n_startup_jobs = 256
 

max_evals = 1024 

space ={
    'preprocessor__numerical__log_transform': hp.choice('preprocessor__numerical__log_transform',
                                                        [LogTransformer(),
                                                         PowerTransformer(method='box-cox')]),
    'preprocessor__numerical__first_scaler': hp.choice('preprocessor__numerical__first_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'preprocessor__numerical__dim_red__n_components': hp.uniform('preprocessor__numerical__dim_red__n_components',
                                                                0.8, 1.0),
    'preprocessor__numerical__second_scaler': hp.choice('preprocessor__numerical__second_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'regression_model__n_estimators': hp.choice('regression_model__n_estimators',
                                                np.arange(100, 1100, 100).tolist()),
    'regression_model__learning_rate': hp.loguniform ('regression_model__learning_rate', 0.01, 0.9),
    'regression_model__base_estimator__max_depth': hp.choice('regression_model__base_estimator__max_depth',
                                                             np.arange(2, 21).tolist()),
    'regression_model__base_estimator__max_leaf_nodes': hp.choice(
                                                        'regression_model__base_estimator__max_leaf_nodes',
                                                        np.arange(5, 31).tolist()),
}

def objective(space):
    adaboost_regression_params = {
        'preprocessor__numerical__log_transform': space['preprocessor__numerical__log_transform'],
        'preprocessor__numerical__first_scaler': space['preprocessor__numerical__first_scaler'],
        'preprocessor__numerical__dim_red__n_components': space['preprocessor__numerical__dim_red__n_components'],
        'preprocessor__numerical__second_scaler': space['preprocessor__numerical__second_scaler'],
        'regression_model__n_estimators': space['regression_model__n_estimators'],
        'regression_model__learning_rate': space['regression_model__learning_rate'],
        'regression_model__base_estimator__max_depth': space['regression_model__base_estimator__max_depth'],
        'regression_model__base_estimator__max_leaf_nodes': space[
                                                            'regression_model__base_estimator__max_leaf_nodes'],
    }
    
    adaboost_regression_pipeline.set_params(**adaboost_regression_params) 
    
    score = - cross_val_score(adaboost_regression_pipeline, X_train, y_train, cv=10,
                              scoring = 'neg_root_mean_squared_error', n_jobs = -1).mean()
    
    return{'loss':score, 'status': STATUS_OK }

trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                   max_evals=max_evals,
                   trials=trials)
 
best_params = space_eval(space, best_params)
print('\nThe best params:')
print ("{:<60} {}".format('Parameter','Selected'))
for k, v in best_params.items():
    print ("{:<60} {}".format(k, v))

adaboost_regression = adaboost_regression_pipeline.set_params(**best_params)
adaboost_regression.fit(X_train, y_train)

In [ ]:
save_model(model = adaboost_regression, name = 'adaboost_regression')
model_evaluation(model = adaboost_regression, name = 'adaboost_regression')

## 10.8 XGBoost Regression

In [ ]:
xgboost_regression_pipeline = Pipeline(steps=[('preprocessor', transformer),
                                          ('regression_model', xgb.XGBRegressor(n_estimators=3, learning_rate=0.5,
                                                                                random_state=123))])

In [ ]:
n_startup_jobs = 512
 

max_evals = 1024 

space ={
    'preprocessor__numerical__log_transform': hp.choice('preprocessor__numerical__log_transform',
                                                        [LogTransformer(),
                                                         PowerTransformer(method='box-cox')]),
    'preprocessor__numerical__first_scaler': hp.choice('preprocessor__numerical__first_scaler',
                                                       [StandardScaler(), MinMaxScaler()]),
    'preprocessor__numerical__dim_red__n_components': hp.uniform('preprocessor__numerical__dim_red__n_components',
                                                                0.8, 1.0),
    'preprocessor__numerical__second_scaler': hp.choice('preprocessor__numerical__second_scaler',
                                                       [None, StandardScaler(), MinMaxScaler()]),
    'regression_model__n_estimators': hp.choice('regression_model__n_estimators',
                                                np.arange(100, 1100, 100).tolist()),
    'regression_model__learning_rate': hp.loguniform ('regression_model__learning_rate', 0.01, 0.5),
    'regression_model__max_depth': hp.choice('regression_model__max_depth', np.arange(2, 11).tolist()),
    'regression_model__min_child_weight': hp.choice('regression_model__min_child_weight',
                                                    np.arange(0, 101).tolist()),
    'regression_model__gamma': hp.loguniform('regression_model__gamma', 0.0, 2.0),
    'regression_model__subsample': hp.uniform('regression_model__subsample', 0.5, 1.0),
    'regression_model__colsample_bytree': hp.uniform('regression_model__colsample_bytree', 0.5, 1.0),
    'regression_model__colsample_bylevel': hp.uniform('regression_model__colsample_bylevel', 0.5, 1.0),
    'regression_model__reg_alpha': hp.loguniform('regression_model__reg_alpha', 0.0, 2.0),
    'regression_model__reg_lambda': hp.loguniform('regression_model__reg_lambda', 0.0, 2.0),
}

def objective(space):
    xgboost_regression_params = {
        'preprocessor__numerical__log_transform': space['preprocessor__numerical__log_transform'],
        'preprocessor__numerical__first_scaler': space['preprocessor__numerical__first_scaler'],
        'preprocessor__numerical__dim_red__n_components': space['preprocessor__numerical__dim_red__n_components'],
        'preprocessor__numerical__second_scaler': space['preprocessor__numerical__second_scaler'],
        'regression_model__n_estimators': space['regression_model__n_estimators'],
        'regression_model__learning_rate': space['regression_model__learning_rate'],
        'regression_model__max_depth': space['regression_model__max_depth'],
        'regression_model__min_child_weight': space['regression_model__min_child_weight'],
        'regression_model__gamma': space['regression_model__gamma'],
        'regression_model__subsample': space['regression_model__subsample'],
        'regression_model__colsample_bytree': space['regression_model__colsample_bytree'],
        'regression_model__colsample_bylevel': space['regression_model__colsample_bylevel'],
        'regression_model__reg_alpha': space['regression_model__reg_alpha'],
        'regression_model__reg_lambda': space['regression_model__reg_lambda'],
    }
    
    xgboost_regression_pipeline.set_params(**xgboost_regression_params) 
    
    score = - cross_val_score(xgboost_regression_pipeline, X_train, y_train, cv=10,
                              scoring = 'neg_root_mean_squared_error', n_jobs = -1).mean()
    
    return{'loss':score, 'status': STATUS_OK }

trials = Trials()
best_params = fmin(fn=objective,
                   space=space,
                   algo=partial(tpe.suggest, n_startup_jobs=n_startup_jobs),
                   max_evals=max_evals,
                   trials=trials)
 
best_params = space_eval(space, best_params)
print('\nThe best params:')
print ("{:<60} {}".format('Parameter','Selected'))
for k, v in best_params.items():
    print ("{:<60} {}".format(k, v))

xgboost_regression = xgboost_regression_pipeline.set_params(**best_params)
xgboost_regression.fit(X_train, y_train)

In [ ]:
save_model(model = xgboost_regression, name = 'xgboost_regression')
model_evaluation(model = xgboost_regression, name = 'xgboost_regression')

# 11. Summary

The results were not satisfactory, which could be due to small data and many variables. It took a lot of time to prepare the scripts to create the data set. You could use a different approach and prepare 9 models, so for each product and area separate. But the models allowed my team to reach the national final.